In [1]:
%pip -q install google-genai

In [2]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [3]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [4]:
from google import genai
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import HTML, display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [5]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [6]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

Projeto de agente para planejamento de passeios
("melhorador de rolês")

In [19]:
##########################################
# --- Agente 1: Processador de input --- #
##########################################
def agente_processador_input(texto_interesses):
  buscador = Agent(
      name='agente_processador_input',
      model='gemini-2.0-flash',
      instruction="""
      Você é um assistente de processamento de texto para identificar palavras chave relacionadas a interesses pessoais que serão
      usadas para buscar locais como restaurantes, bares, museus, parques, exposições, construções, cinemas, teatros.
      A sua tarefa é identificar as palavas chave do texto abaixo que estejam mais potencialmente relacionadas
      a esses tipos de estabelecimentos.
      Ao identificar as palavras chave, você deve criar uma lista que contenha apenas as palavras chave
      extraídas do texto e responder somente com a lista.
      Exemplo de resposta: [palavra_chave_1, palavra_chave_2, palavra_chave_3].
      """,
      description='Agente processador de informações de input para extrair palavras chave relacionadas a interesses pessoais.',
      tools=[google_search]
  )

  entrada_do_agente_processador_input= f"Texto: {texto_interesses}"

  lancamentos = call_agent(buscador, entrada_do_agente_processador_input)
  return lancamentos

In [33]:
############################################
# --- Agente 2: Pesquisador geográfico --- #
############################################
def agente_pesquisador_geográfico(data_evento, local_inicial):
  buscador = Agent(
      name='agente_pesquisador_geográfico',
      model='gemini-2.0-flash',
      instruction="""
      Você é um profundo conhecedor de restaurantes, bares, museus, parques, exposições, construções, cinemas e teatros e está sempre
      atento a novos locais que são abertos e a novas exposições, peças e filmes em exibição. Sua tarefa é utilizar a ferramenta (google_maps)
      para identificar a região onde se localiza um certo estabelecimento fornecido abaixo. Identifique a cidade e o bairro
      onde esse estabelecimento está localizado.
      A partir disso, utilize a cidade identificada, regiões próximas ao bairro identificado (com distãncia de até cerca de 3km)
      e a data fornecida abaixo para pesquisar no (google_search) eventos e locais para serem visitados.
      Procure por eventos atuais e que estejam abertos e disponíveis na data fornecida e por locais que sejam bem avaliados
      e que tenham grande quantidade de comentários positivos.
      Você deve retornar uma lista de locais que possam potencialmente ser visitados na região e dia determinados.
      Retorne apenas o nome do local e o tipo de local em uma lista com o seguinte formato: [(local1, tipo_local1), (local2, tipo_local2)].
      """,
      # Realize essas tarefas passo a passo considerando todas as variáveis (cidade, bairros próximos e data).
      description='Agente que pesquisa estabelecimentos a serem visitados em área e data específica.',
      tools=[google_search]
  )

  entrada_do_agente_pesquisador_geográfico= f"Estabelecimento: {local_inicial}\nData: {data_evento}"

  lancamentos = call_agent(buscador, entrada_do_agente_pesquisador_geográfico)
  return lancamentos

In [39]:
############################################
# --- Agente 3: Avaliador de locais --- #
############################################
def agente_avaliador_locais(local_inicial, locais_pesquisados, lista_interesses):
  buscador = Agent(
      name='agente_avaliador_locais',
      model='gemini-2.0-flash',
      instruction="""
      Você é um especialista em planejamento de eventos e gestão de projetos.
      Sua tarefa é avaliar uma série de locais selecionados e classificá-los de acordo com a proximidade de um local inicial e
      de acordo com a afinidade com uma lista de interesses.
      Primeiro ocê deve pegar cada item da lista de locais pesquisados e utilizar o (google_maps) para obter a distância
      em kilometros entre esse local e o local inicial.
      Depois você deve usar o (google_search) para cada local da lista e elaborar uma descrição de cada local
      com foco no que pode ser feito no local e no tipo de pessoas que o frequentam.
      Na sequência, analise a descrição de cada local com relação à lista de interesses pessoais abaixo e dê uma nota de 0 a 10 para
      o quanto cada local se adequa aos interesses presentes.
      Ao final elabore uma lista em que constem apenas os locais com nota superior a 5 e distância menor que 3km.
      """,
      description='Agente que pega cada local pesquisado, determina a distância a partir do local inicial e avalia cada local com relação aos interesses pessoais.',
      tools=[google_search]
  )

  entrada_do_agente_avaliador_locais= f"Local inicial: {local_inicial}\nLocais selecionados: {locais_pesquisados}\nLista de interesses: {lista_interesses}"

  lancamentos = call_agent(buscador, entrada_do_agente_avaliador_locais)
  return lancamentos

In [37]:
interesses_identificados = agente_processador_input(texto)
print("\n--- Resultado do agente 1 (processador de texto) ---\n")
display(to_markdown(interesses_identificados))
print("--------------------------------------------------------")


--- Resultado do agente 1 (processador de texto) ---



> [arquitetura, urbanismo, plantas, história, cultura, lugares calmos, decoração, comidinhas apetitosas]
> 


--------------------------------------------------------


In [34]:
locais_pesquisados = agente_pesquisador_geográfico(date.today(), local)
print("\n--- Resultado do agente 2 (pesquisador geográfico) ---\n")
display(to_markdown(locais_pesquisados))
print("--------------------------------------------------------")


--- Resultado do agente 2 (pesquisador geográfico) ---



> O restaurante Pastissimo Vila Madalena está localizado em São Paulo, no bairro de Sumaré, próximo à Vila Madalena.
> 
> Aqui está uma lista de locais e eventos que podem ser visitados em São Paulo, nas proximidades da Vila Madalena, no dia 18 de maio de 2025:
> 
> *   **Festival Pinheiros:** (evento) Gastronomia, cultura e diversão na Rua dos Pinheiros, Pinheiros. Entrada gratuita.
> *   **Beco do Batman:** (ponto turístico) Famoso pela arte de rua e grafites.
> *   **Parque Villa-Lobos:** (parque) Amplo espaço de lazer com gramados, quadras, ciclovia, parquinho e Biblioteca Parque Villa-Lobos.
> *   **Praça do Pôr do Sol:** (praça) Conhecida pela vista panorâmica do pôr do sol.
> *   **Natan por aí e a Lenda dos Gigantes:** (teatro) Apresentação no Teatro Bradesco, no bairro de Perdizes.
> *   **Experimentos do Invisível:** (evento) Localizado em Perdizes.
> *   **Vai de Roteiro - Vila Madalena:** (passeio guiado) Tour guiado gratuito pela Vila Madalena, com saída da Rua Pinheiros.
> *   **Run das Mães - Corrida e Caminhada:** (corrida) Evento de corrida no Parque Bruno Covas.
> *   **Feira de Artesanato Parque Cordeiro - Martin Luther King:** (feira de artesanato) Ocorre no Parque Cordeiro.
> *   **Feira de Artesanato Parque Jardim das Perdizes:** (feira de artesanato) Ocorre no Parque Jardim das Perdizes.
> *   **Feira Parque Buenos Aires:** (feira de artesanato) Ocorre na Av. Angélica, 1500, Higienópolis.
> *   **YIN WAR 1ST FAN MEETING:** (evento) Informações sobre o evento podem ser encontradas no site Visite São Paulo.
> *   **CONCERTO PAOLO ANDRÉ GUALDI:** (concerto) Informações sobre o evento podem ser encontradas no site Visite São Paulo.
> *   **BESOURO MULHER E PELADOS:** (evento) Informações sobre o evento podem ser encontradas no site Visite São Paulo.
> *   **PATRULHA SHOW EM AVENTURAS CANINAS:** (evento) Informações sobre o evento podem ser encontradas no site Visite São Paulo.
> *    **Vila Madalena:** (bares) Bares como o "De Primeira Botequim", "Bar São Cristóvão", "Bar Alto", "Madeleine Bar", "O Pasquim Bar e Prosa", "Boteco Todos os Santos", "O SubAstor" e "Olívio Bar".
> 
> **Museus próximos:**
> 
> *   **Galeria Raquel Arnaud:** (galeria de arte) Focada em arte contemporânea.
> *   **Museu de Arte de São Paulo (MASP):** (museu) Um dos museus mais importantes da América Latina, localizado na Avenida Paulista.
> *   **Pinacoteca do Estado:** (museu) Possui uma extensa coleção de arte brasileira.
> 


--------------------------------------------------------


In [40]:
locais_avaliados = agente_avaliador_locais(local, locais_pesquisados, interesses_identificados)
print("\n--- Resultado do agente 3 (avaliador de locais) ---\n")
display(to_markdown(locais_avaliados))
print("--------------------------------------------------------")


--- Resultado do agente 3 (avaliador de locais) ---



> Okay, vamos lá. Primeiro, vou coletar as distâncias de cada local em relação ao "Pastissimo Vila Madalena" e, em seguida, pesquisar informações sobre cada um para avaliar sua adequação aos seus interesses.
> 
> 
> Ok, coletei as informações. Agora vou analisar cada local e atribuir as notas.
> 
> **Festival Pinheiros:**
> 
> *   **Distância:** Próximo ao Pastissimo Vila Madalena (localizado na Rua dos Pinheiros).
> *   **Descrição:** Evento de rua com gastronomia, cultura, lazer e sustentabilidade. Música ao vivo, atividades para crianças e pets, comidinhas artesanais e atrações para toda a família.
> *   **Afinidade com Interesses:** Gastronomia (comidinhas apetitosas), cultura (música, arte de rua), urbanismo (evento de rua revitalizando o espaço público).
> *   **Nota:** 8/10
> 
> **Beco do Batman:**
> 
> *   **Distância:** Próximo ao Pastissimo Vila Madalena (Vila Madalena).
> *   **Descrição:** Galeria de arte urbana a céu aberto, com grafites e murais em constante renovação. Lojas de artesanato, bares e restaurantes nas proximidades.
> *   **Afinidade com Interesses:** Arquitetura/Urbanismo (arte de rua transformando o espaço), cultura (grafite, arte urbana), decoração (murais), história (evolução da arte no local).
> *   **Nota:** 9/10
> 
> **Parque Villa-Lobos:**
> 
> *   **Distância:** Aproximadamente 4.8 km do Pastissimo Vila Madalena.
> *   **Descrição:** Amplo espaço de lazer com gramados, quadras, ciclovia, parquinho, Biblioteca Parque Villa-Lobos e Orquidário Ruth Cardoso. Espécies nativas da Mata Atlântica.
> *   **Afinidade com Interesses:** Plantas (Mata Atlântica, orquidário), urbanismo (espaço de lazer), lugares calmos (gramados), arquitetura (circuito das árvores).
> *   **Nota:** 7/10
> 
> **Praça do Pôr do Sol:**
> 
> *   **Distância:** Aproximadamente 2.1 km do Pastissimo Vila Madalena.
> *   **Descrição:** Praça com vista panorâmica do pôr do sol, ideal para piqueniques, meditação e atividades ao ar livre.
> *   **Afinidade com Interesses:** Lugares calmos (meditação), plantas (árvores remanescentes da Mata Atlântica), arquitetura/urbanismo (vista panorâmica).
> *   **Nota:** 8/10
> 
> **Natan por aí e a Lenda dos Gigantes (Teatro Bradesco):**
> 
> *   **Distância:** Aproximadamente 3.8 km do Pastissimo Vila Madalena.
> *   **Descrição:** Teatro localizado em Perdizes, com apresentações variadas.
> *   **Afinidade com Interesses:** Cultura (teatro). Depende do espetáculo.
> *   **Nota:** 5/10 (genérico, sem saber o conteúdo do espetáculo)
> 
> **Experimentos do Invisível (Perdizes):**
> 
> *   **Distância:** Aproximadamente 3.8 km do Pastissimo Vila Madalena.
> *   **Descrição:** Informações limitadas. Necessário mais detalhes sobre o evento.
> *   **Afinidade com Interesses:** Impossível avaliar sem mais informações.
> *   **Nota:** 2/10 (sem informações)
> 
> **Vai de Roteiro - Vila Madalena:**
> 
> *   **Distância:** O ponto de saída (Rua Pinheiros) é próximo ao Pastissimo Vila Madalena.
> *   **Descrição:** Tour guiado gratuito pela Vila Madalena.
> *   **Afinidade com Interesses:** História (do bairro), cultura (exploração local), arquitetura/urbanismo (conhecer a Vila Madalena).
> *   **Nota:** 7/10
> 
> **Run das Mães - Corrida e Caminhada (Parque Bruno Covas):**
> 
> *   **Distância:** Aproximadamente 6.5 km do Pastissimo Vila Madalena.
> *   **Descrição:** Evento de corrida.
> *   **Afinidade com Interesses:** Nenhum dos interesses listados.
> *   **Nota:** 1/10
> 
> **Feira de Artesanato Parque Cordeiro - Martin Luther King:**
> 
> *   **Distância:** Aproximadamente 11 km do Pastissimo Vila Madalena.
> *   **Descrição:** Feira de artesanato.
> *   **Afinidade com Interesses:** Decoração (artesanato).
> *   **Nota:** 4/10
> 
> **Feira de Artesanato Parque Jardim das Perdizes:**
> 
> *   **Distância:** Aproximadamente 3.8 km do Pastissimo Vila Madalena.
> *   **Descrição:** Feira de artesanato. O parque conta com pista de cooper, ciclovia, playground, equipamentos de ginástica para a terceira idade e administração.
> *   **Afinidade com Interesses:** Decoração (artesanato).
> *   **Nota:** 4/10
> 
> **Feira Parque Buenos Aires:**
> 
> *   **Distância:** Aproximadamente 5.2 km do Pastissimo Vila Madalena.
> *   **Descrição:** Feira de artesanato.
> *   **Afinidade com Interesses:** Decoração (artesanato).
> *   **Nota:** 4/10
> 
> **YIN WAR 1ST FAN MEETING:**
> 
> *   **Distância:** Não foi possível determinar a localização exata.
> *   **Descrição:** Evento. Necessita de mais informações.
> *   **Afinidade com Interesses:** Não é possível avaliar.
> *   **Nota:** 1/10
> 
> **CONCERTO PAOLO ANDRÉ GUALDI:**
> 
> *   **Distância:** Não foi possível determinar a localização exata.
> *   **Descrição:** Concerto. Necessita de mais informações.
> *   **Afinidade com Interesses:** Cultura (música).
> *   **Nota:** 4/10 (genérico, dependendo do estilo musical)
> 
> **BESOURO MULHER E PELADOS:**
> 
> *   **Distância:** Não foi possível determinar a localização exata.
> *   **Descrição:** Evento. Necessita de mais informações.
> *   **Afinidade com Interesses:** Não é possível avaliar.
> *   **Nota:** 1/10
> 
> **PATRULHA SHOW EM AVENTURAS CANINAS:**
> 
> *   **Distância:** Não foi possível determinar a localização exata.
> *   **Descrição:** Evento. Necessita de mais informações.
> *   **Afinidade com Interesses:** Não é possível avaliar.
> *   **Nota:** 1/10
> 
> **Vila Madalena (bares):**
> 
> *   **Distância:** Próximo ao Pastissimo Vila Madalena.
> *   **Descrição:** Diversos bares com diferentes estilos, como "De Primeira Botequim" (ambiente alegre, samba), "Bar São Cristóvão" (temático de futebol), "Bar Alto" (cozinha brasileira), "Madeleine Bar" (jazz e gastronomia criativa), "O Pasquim Bar e Prosa" (música ao vivo, ambiente descontraído), "Boteco Todos os Santos" (samba e feijoada), "O SubAstor" (coquetelaria) e "Olívio Bar" (drinks temáticos).
> *   **Afinidade com Interesses:** Cultura (música ao vivo, samba), comidinhas apetitosas (petiscos de bar), decoração (ambiente dos bares), lugares calmos (dependendo do bar).
> *   **Nota:** 7/10 (média geral, alguns bares podem ter notas maiores dependendo do estilo)
> 
> **Galeria Raquel Arnaud:**
> 
> *   **Distância:** Próximo ao Pastissimo Vila Madalena (Vila Madalena).
> *   **Descrição:** Galeria de arte focada em arte construtiva e cinética, com obras de artistas renomados. Espaços assinados por arquitetos como Lina Bo Bardi e Ruy Ohtake.
> *   **Afinidade com Interesses:** Arquitetura (espaço da galeria), cultura (arte contemporânea), decoração (esculturas).
> *   **Nota:** 8/10
> 
> **Museu de Arte de São Paulo (MASP):**
> 
> *   **Distância:** Aproximadamente 6.4 km do Pastissimo Vila Madalena.
> *   **Descrição:** Um dos museus mais importantes da América Latina, com acervo de arte brasileira e internacional, exposições temporárias e arquitetura icônica.
> *   **Afinidade com Interesses:** Arquitetura (prédio do museu), cultura (arte), história (acervo).
> *   **Nota:** 9/10
> 
> **Pinacoteca do Estado:**
> 
> *   **Distância:** Aproximadamente 7.8 km do Pastissimo Vila Madalena.
> *   **Descrição:** Possui uma extensa coleção de arte brasileira.
> *   **Afinidade com Interesses:** Cultura (arte brasileira), história (acervo).
> *   **Nota:** 8/10
> 
> **Lista Final (Nota > 5 e Distância < 3km):**
> 
> *   **Festival Pinheiros**
> *   **Beco do Batman**
> *   **Praça do Pôr do Sol**
> *   **Vai de Roteiro - Vila Madalena**
> *   **Vila Madalena (bares)**
> *   **Galeria Raquel Arnaud**
> 
> Espero que esta análise detalhada te ajude a planejar um ótimo dia na Vila Madalena e arredores!
> 


--------------------------------------------------------


In [ ]:
print("🚀 Iniciando o Sistema de sugestão de passeios! 🚀")

# --- Obter os inputs do Usuário ---
data_evento = date.today()
local_inicial = input("Olá! Eu vou te ajudar a encontrar locais para compor e melhorar seu passeio de hoje!\nMe conte aonde você quer ir hoje e eu vou encontrar locais próximos para que você aproveite ainda mais o seu dia!\n Em qual local você irá hoje?: ")

# Inserir lógica do sistema de agentes ################################################
if not local_inicial:
  print('Escolha um local!')

texto_interesses = input('Agora me conte quais são seus interesses (locais em que gosta de ir, coisas que gosta de fazer ou comer, tipos de arte que te atraem): ')
if not texto_interesses:
  print('Defina seus interesses!')
else:
  print(f"Perfeito! Vamos criar uma lista de locais para tornar seu passeio ainda mais interessante!")

  interesses_identificados = agente_processador_input(texto)
  print("\n--- Resultado do agente 1 (processador de texto) ---\n")
  display(to_markdown(interesses_identificados))
  print("--------------------------------------------------------")

  locais_pesquisados = agente_pesquisador_geográfico(date.today(), local)
  print("\n--- Resultado do agente 2 (pesquisador geográfico) ---\n")
  display(to_markdown(locais_pesquisados))
  print("--------------------------------------------------------")

  locais_avaliados = agente_avaliador_locais(local, locais_pesquisados, interesses_identificados)
  print("\n--- Resultado do agente 3 (avaliador de locais) ---\n")
  display(to_markdown(locais_avaliados))
  print("--------------------------------------------------------")